# Обработка данных по объемам

Обработка объемных данных для выявления дней повышенной волатильности. Идея в том, чтобы найти дни в которые проходят большие объемы и есть идея направления движения.

In [109]:
import sqlite3
import pandas as pd
import numpy as np

In [110]:
# Установить соединение с базой данных
conn = sqlite3.connect(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_minute.db')

# SQL-запрос для извлечения данных
query = "SELECT TRADEDATE, VOLUME FROM Minute"

# Загрузить данные в DataFrame
df = pd.read_sql_query(query, conn)

# Закрыть соединение
conn.close()

df

,TRADEDATE,VOLUME
0,2015-01-05 10:00:00,5970
1,2015-01-05 10:01:00,3455
2,2015-01-05 10:02:00,2721
3,2015-01-05 10:03:00,4008
4,2015-01-05 10:04:00,1471
...,...,...
2024358,2024-12-06 23:45:00,34
2024359,2024-12-06 23:46:00,168
2024360,2024-12-06 23:47:00,32
2024361,2024-12-06 23:48:00,18


In [111]:
df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])

# Добавление колонки с датой
df['date'] = df['TRADEDATE'].dt.date

# Добавление колонки с временем
df['time'] = df['TRADEDATE'].dt.time

# # Вычисление накопленных объемов по дням
# df['daily_cumulative_volume'] = df.groupby('date')['VOLUME'].cumsum()

df

,TRADEDATE,VOLUME,date,time
0,2015-01-05 10:00:00,5970,2015-01-05,10:00:00
1,2015-01-05 10:01:00,3455,2015-01-05,10:01:00
2,2015-01-05 10:02:00,2721,2015-01-05,10:02:00
3,2015-01-05 10:03:00,4008,2015-01-05,10:03:00
4,2015-01-05 10:04:00,1471,2015-01-05,10:04:00
...,...,...,...,...
2024358,2024-12-06 23:45:00,34,2024-12-06,23:45:00
2024359,2024-12-06 23:46:00,168,2024-12-06,23:46:00
2024360,2024-12-06 23:47:00,32,2024-12-06,23:47:00
2024361,2024-12-06 23:48:00,18,2024-12-06,23:48:00


In [112]:
# # Вычисление перцентилей за последние 30 дней
# def calculate_percentile(row, window):
#     volumes = window['daily_cumulative_volume']
#     return np.sum(volumes < row['daily_cumulative_volume']) / len(volumes) * 100

# # Группировка по 30-дневным окнам и вычисление перцентилей
# df['percentile'] = (
#     df['daily_cumulative_volume']
#     .rolling(window=30, min_periods=1)  # 30-дневное окно
#     .apply(lambda x: np.sum(x < x.iloc[-1]) / len(x) * 100, raw=False)  # Процентиль текущего дня
# )

# print(df)

Создаем колонку с перцентилем объема к выборке за 30 последних значений с таким же временем.

In [113]:
# Группировка по времени
df["percentile"] = (
    df.groupby("time")["VOLUME"]
    .rolling(window=30, min_periods=1)  # Используем скользящее окно
    .apply(lambda x: (x <= x.iloc[-1]).mean() * 100)  # Рассчёт перцентиля
    .reset_index(level=0, drop=True)  # Сбрасываем лишний индекс
)

Создаем колонку со средними значениями предыдущих строк колонки percentile за каждый день

In [114]:
# Группировка по дате и расчёт среднего
df["avg_percentile"] = (
    df.groupby("date")["percentile"]  # Группировка по дате
    # .shift()  # Если текущая строка не должна учитываться при расчёте среднего (сдвиг)
    .expanding()  # Нарастающее среднее внутри каждой группы
    .mean()  # Среднее
    .reset_index(level=0, drop=True)  # Сбрасываем лишний индекс
)

In [115]:
# Округление значений в указанных колонках до двух знаков
df[["percentile", "avg_percentile"]] = df[["percentile", "avg_percentile"]].round(2)

print(df)

                  TRADEDATE  VOLUME        date      time  percentile  \
0       2015-01-05 10:00:00    5970  2015-01-05  10:00:00      100.00   
1       2015-01-05 10:01:00    3455  2015-01-05  10:01:00      100.00   
2       2015-01-05 10:02:00    2721  2015-01-05  10:02:00      100.00   
3       2015-01-05 10:03:00    4008  2015-01-05  10:03:00      100.00   
4       2015-01-05 10:04:00    1471  2015-01-05  10:04:00      100.00   
...                     ...     ...         ...       ...         ...   
2024358 2024-12-06 23:45:00      34  2024-12-06  23:45:00       33.33   
2024359 2024-12-06 23:46:00     168  2024-12-06  23:46:00       93.33   
2024360 2024-12-06 23:47:00      32  2024-12-06  23:47:00       50.00   
2024361 2024-12-06 23:48:00      18  2024-12-06  23:48:00       30.00   
2024362 2024-12-06 23:49:00      68  2024-12-06  23:49:00       56.67   

         avg_percentile  
0                100.00  
1                100.00  
2                100.00  
3                10

In [116]:
# # Запись в Excel
# df.head(2000).to_excel('example.xlsx')

# Сохраняем первые строки в CSV
df.iloc[:3000].to_csv("head_df.csv", index=False)

# Сохраняем последние 3000 строк в CSV
df.tail(3000).to_csv("last_rows.csv", index=False)

Создаем список из значений колонки date где значение avg_percentile на 60 строке за дату больше либо равно 75.

In [117]:
# Фильтрация дат, где значение avg_percentile на 60-й строке >= 75
dates_list = (
    df.groupby("date")  # Группировка по дате
    .nth(59)  # Выбор 60-й строки (индексация с нуля)
    .query("avg_percentile >= 75")  # Фильтр по условию
    # .index.tolist()  # Преобразование индекса в список
    .date.tolist()  # Преобразование дат в список
)
unique_values = df["date"].nunique()

print("Список дат:", dates_list)
print("Длина списка:", len(dates_list))
print(f"Уникальные значения в колонке date: {unique_values}")
print(len(dates_list) / unique_values)

Список дат: [datetime.date(2015, 1, 5), datetime.date(2015, 1, 26), datetime.date(2015, 2, 6), datetime.date(2015, 2, 9), datetime.date(2015, 2, 12), datetime.date(2015, 3, 26), datetime.date(2015, 4, 9), datetime.date(2015, 4, 10), datetime.date(2015, 4, 13), datetime.date(2015, 6, 22), datetime.date(2015, 7, 8), datetime.date(2015, 7, 28), datetime.date(2015, 8, 3), datetime.date(2015, 8, 24), datetime.date(2015, 9, 1), datetime.date(2016, 1, 18), datetime.date(2016, 1, 19), datetime.date(2016, 1, 20), datetime.date(2016, 1, 21), datetime.date(2016, 1, 22), datetime.date(2016, 1, 25), datetime.date(2016, 1, 29), datetime.date(2016, 3, 17), datetime.date(2016, 4, 13), datetime.date(2016, 4, 18), datetime.date(2016, 4, 20), datetime.date(2016, 4, 21), datetime.date(2016, 11, 9), datetime.date(2016, 12, 12), datetime.date(2017, 2, 28), datetime.date(2017, 3, 9), datetime.date(2017, 3, 10), datetime.date(2017, 11, 8), datetime.date(2018, 1, 18), datetime.date(2018, 1, 25), datetime.date(